# Tiền Xử Lý Dữ Liệu Bất Động Sản Nhatot
## Data Preprocessing for Nhatot Housing Dataset

Notebook này thực hiện các bước:
1. 📂 Load dữ liệu
2. 🧹 Làm sạch dữ liệu
3. 🔍 Phân tích missing values
4. 🔧 Xử lý missing values
5. 🏷️ Encoding categorical features
6. ⚙️ Feature engineering
7. 📏 Scaling features (optional)
8. 💾 Lưu dữ liệu đã xử lý

**Lưu ý:** File gốc `nhatot_crawl4ai.csv` sẽ KHÔNG bị thay đổi!

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Libraries imported successfully!")

## 2. Load Dữ Liệu

In [ ]:
# Load dữ liệu
file_path = 'nhatot_crawl4ai.csv'
df = pd.read_csv(file_path)

# Tạo bản sao để bảo toàn dữ liệu gốc
df_original = df.copy()

print(f"📂 Loaded data:")
print(f"   Rows: {len(df):,}")
print(f"   Columns: {len(df.columns)}")
print(f"   ✓ Created backup of original data")
print(f"\nColumns: {list(df.columns)}")

In [ ]:
# Xem 5 dòng đầu tiên
df.head()

In [ ]:
# Thông tin tổng quan
df.info()

## 3. Làm Sạch Dữ Liệu

### 3.1. Xóa Các Dòng Trống Hoàn Toàn

In [ ]:
print("🧹 Cleaning empty rows...")
initial_count = len(df)

# Xóa dòng có tất cả giá trị là NaN
df = df.dropna(how='all')

removed = initial_count - len(df)
print(f"✓ Removed {removed:,} empty rows")
print(f"  Remaining: {len(df):,} rows")

### 3.2. Xóa Dữ Liệu Trùng Lặp

In [ ]:
print("🔍 Removing duplicates...")
initial_count = len(df)

df = df.drop_duplicates()

removed = initial_count - len(df)
print(f"✓ Removed {removed:,} duplicate rows")
print(f"  Remaining: {len(df):,} rows")

## 4. Phân Tích Missing Values

In [ ]:
# Phân tích missing values
missing_stats = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values(
    'Missing_Percentage', ascending=False
)

print("📊 Missing Values Analysis:")
print("=" * 60)
print(missing_stats.to_string(index=False))
print("=" * 60)

In [ ]:
# Visualize missing values
plt.figure(figsize=(12, 6))
if len(missing_stats) > 0:
    sns.barplot(data=missing_stats, x='Column', y='Missing_Percentage', palette='viridis')
    plt.xticks(rotation=45, ha='right')
    plt.title('Missing Values by Column (%)', fontsize=14, fontweight='bold')
    plt.ylabel('Missing Percentage (%)')
    plt.xlabel('Column Name')
    plt.tight_layout()
    plt.show()
else:
    print("No missing values found!")

## 5. Xử Lý Cột Giá (Price Column)

In [ ]:
def parse_price(price_str):
    """
    Convert Vietnamese price format to numeric value
    
    Examples:
        "1,5 tỷ" -> 1500000000
        "500 triệu" -> 500000000
    """
    if pd.isna(price_str):
        return np.nan
    
    price_str = str(price_str).strip().replace('"', '')
    
    try:
        if 'tỷ' in price_str:
            value = price_str.replace('tỷ', '').strip().replace(',', '.')
            return float(value) * 1_000_000_000
        elif 'triệu' in price_str:
            value = price_str.replace('triệu', '').strip().replace(',', '.')
            return float(value) * 1_000_000
        else:
            value = price_str.replace(',', '.')
            return float(value)
    except:
        return np.nan

# Áp dụng conversion
print("💰 Processing price column...")
df['Giá bán (VND)'] = df['Giá bán'].apply(parse_price)

# Xóa các dòng có giá không hợp lệ
initial_count = len(df)
df = df.dropna(subset=['Giá bán (VND)'])
removed = initial_count - len(df)

print(f"✓ Converted prices to numeric")
print(f"  Removed {removed:,} rows with invalid prices")
print(f"  Price range: {df['Giá bán (VND)'].min():,.0f} - {df['Giá bán (VND)'].max():,.0f} VND")
print(f"  Mean price: {df['Giá bán (VND)'].mean():,.0f} VND")

In [ ]:
# Visualize price distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['Giá bán (VND)'] / 1e9, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Giá (tỷ VND)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Price Distribution')

# Box plot
axes[1].boxplot(df['Giá bán (VND)'] / 1e9)
axes[1].set_ylabel('Giá (tỷ VND)')
axes[1].set_title('Price Box Plot')

plt.tight_layout()
plt.show()

## 6. Xử Lý Các Cột Số (Numeric Columns)

In [ ]:
def parse_numeric_column(val):
    """Parse numeric columns that might have special values"""
    if pd.isna(val):
        return np.nan
    
    val_str = str(val).strip().lower()
    
    # Handle special values like "nhiều hơn 6"
    if 'nhiều hơn' in val_str or 'hơn' in val_str:
        import re
        numbers = re.findall(r'\d+', val_str)
        if numbers:
            return float(numbers[0]) + 1
        return np.nan
    
    try:
        return float(val_str.replace(',', '.'))
    except:
        return np.nan

# Áp dụng cho các cột số
print("🔢 Processing numeric columns...")
numeric_cols = [
    'Diện tích (m2)',
    'Chiều ngang (m)',
    'Chiều dài (m)',
    'Số phòng ngủ',
    'Số phòng vệ sinh',
    'Số tầng'
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].apply(parse_numeric_column)
        print(f"  ✓ Cleaned {col}")

print("✓ All numeric columns processed")

## 7. Xử Lý Missing Values

### Chiến lược:
- **Numeric columns**: Điền bằng median
- **Categorical columns**: Điền bằng mode hoặc "Không rõ"

In [ ]:
print("🔧 Handling missing values...")

# 1. Numeric columns: fill with median
numeric_cols_in_df = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols_in_df:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)
        print(f"  ✓ Filled {col} with median: {median_val:.2f}")

# 2. Categorical columns: fill with mode or 'Không rõ'
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col == 'Giá bán':  # Skip original price column
        continue
    if df[col].isnull().sum() > 0:
        mode_val = df[col].mode()
        if len(mode_val) > 0:
            df[col].fillna(mode_val[0], inplace=True)
            print(f"  ✓ Filled {col} with mode: {mode_val[0]}")
        else:
            df[col].fillna('Không rõ', inplace=True)
            print(f"  ✓ Filled {col} with 'Không rõ'")

print(f"\n✓ Missing values handled. Remaining rows: {len(df):,}")

In [ ]:
# Kiểm tra lại missing values
remaining_missing = df.isnull().sum().sum()
print(f"Remaining missing values: {remaining_missing}")

if remaining_missing > 0:
    print("\nColumns still have missing values:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

## 8. Feature Engineering

Tạo các features mới từ dữ liệu hiện có

In [ ]:
print("⚙️  Feature Engineering...")

# 1. Price per square meter
if 'Diện tích (m2)' in df.columns and 'Giá bán (VND)' in df.columns:
    df['Giá/m2'] = df['Giá bán (VND)'] / df['Diện tích (m2)']
    print("  ✓ Created 'Giá/m2' (price per sqm)")

# 2. Total rooms
if 'Số phòng ngủ' in df.columns and 'Số phòng vệ sinh' in df.columns:
    df['Tổng số phòng'] = df['Số phòng ngủ'] + df['Số phòng vệ sinh']
    print("  ✓ Created 'Tổng số phòng' (total rooms)")

# 3. Area from dimensions
if 'Chiều ngang (m)' in df.columns and 'Chiều dài (m)' in df.columns:
    df['Diện tích ước tính'] = df['Chiều ngang (m)'] * df['Chiều dài (m)']
    print("  ✓ Created 'Diện tích ước tính' (estimated area)")

# 4. Property size category
if 'Diện tích (m2)' in df.columns:
    def categorize_size(area):
        if pd.isna(area):
            return 'Không rõ'
        if area < 30:
            return 'Rất nhỏ'
        elif area < 50:
            return 'Nhỏ'
        elif area < 80:
            return 'Trung bình'
        elif area < 150:
            return 'Lớn'
        else:
            return 'Rất lớn'
    
    df['Kích thước'] = df['Diện tích (m2)'].apply(categorize_size)
    print("  ✓ Created 'Kích thước' (size category)")

print("\n✓ Feature engineering completed!")

In [ ]:
# Visualize new features
if 'Kích thước' in df.columns:
    plt.figure(figsize=(10, 6))
    size_counts = df['Kích thước'].value_counts()
    plt.pie(size_counts, labels=size_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title('Distribution by Property Size', fontsize=14, fontweight='bold')
    plt.axis('equal')
    plt.show()

## 9. Encoding Categorical Features

Chuyển đổi các biến categorical thành dạng số

In [ ]:
print("🏷️  Encoding categorical features...")

categorical_cols = [
    'Thành phố',
    'Phường/Xã',
    'Loại hình',
    'Giấy tờ pháp lý',
    'Hướng',
    'Tình trạng nội thất'
]

label_encoders = {}

for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        # Handle NaN by treating as a separate category
        df[col] = df[col].fillna('Không rõ')
        df[f'{col}_encoded'] = le.fit_transform(df[col])
        label_encoders[col] = le
        n_categories = len(le.classes_)
        print(f"  ✓ Encoded {col} ({n_categories} categories)")

print("\n✓ All categorical features encoded!")

In [ ]:
# Xem mapping của một vài categorical features
print("\nExample: Loại hình encoding")
if 'Loại hình' in df.columns:
    mapping_df = pd.DataFrame({
        'Original': label_encoders['Loại hình'].classes_,
        'Encoded': range(len(label_encoders['Loại hình'].classes_))
    })
    print(mapping_df)

## 10. Scaling Features (Optional)

**Lưu ý:** Bạn có thể bỏ qua bước này nếu không cần scaling

In [ ]:
# Bỏ comment để chạy scaling
# print("📏 Scaling features...")

# scaler = StandardScaler()
# cols_to_scale = ['Diện tích (m2)', 'Chiều ngang (m)', 'Chiều dài (m)', 'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng']

# for col in cols_to_scale:
#     if col in df.columns:
#         df[f'{col}_scaled'] = scaler.fit_transform(df[[col]])
#         print(f"  ✓ Scaled {col}")

# print("\n✓ Features scaled!")

## 11. Summary Statistics

In [ ]:
print("📈 Summary Statistics:")
print("=" * 80)

# Numeric features
print("\nNumeric Features:")
numeric_cols_display = df.select_dtypes(include=[np.number]).columns
df[numeric_cols_display].describe()

In [ ]:
# Categorical features
print("\nCategorical Features (Top 5 values):")
categorical_cols_display = ['Thành phố', 'Loại hình', 'Giấy tờ pháp lý', 'Tình trạng nội thất']

for col in categorical_cols_display:
    if col in df.columns:
        print(f"\n{col}:")
        print(df[col].value_counts().head())

## 12. Visualization: Correlation Matrix

In [ ]:
# Correlation matrix
plt.figure(figsize=(14, 10))

# Select key numeric columns
key_cols = ['Giá bán (VND)', 'Diện tích (m2)', 'Chiều ngang (m)', 'Chiều dài (m)', 
            'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng', 'Giá/m2', 'Tổng số phòng']
key_cols = [col for col in key_cols if col in df.columns]

corr_matrix = df[key_cols].corr()

sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Key Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

## 13. Lưu Dữ Liệu Đã Xử Lý

In [ ]:
# Lưu dữ liệu đã xử lý
output_file = 'nhatot_crawl4ai_processed.csv'
backup_file = 'nhatot_crawl4ai_original_backup.csv'

print("💾 Saving files...")

# Save processed data
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"✓ Processed data: {output_file}")
print(f"  → {len(df):,} rows and {len(df.columns)} columns")

# Save original data backup
df_original.to_csv(backup_file, index=False, encoding='utf-8-sig')
print(f"\n✓ Original data backup: {backup_file}")
print(f"  → {len(df_original):,} rows and {len(df_original.columns)} columns")

print(f"\n📁 Files saved:")
print(f"  • Original file (unchanged): {file_path}")
print(f"  • Processed file (new): {output_file}")
print(f"  • Backup file (new): {backup_file}")

## 14. Chuẩn Bị Cho Machine Learning (Optional)

In [ ]:
# Prepare for modeling
print("🎯 Preparing data for modeling...")

# Select feature columns (encoded and numeric)
feature_cols = []
for col in df.columns:
    if '_encoded' in col:
        feature_cols.append(col)
    elif col in ['Diện tích (m2)', 'Chiều ngang (m)', 'Chiều dài (m)', 
                 'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng', 'Tổng số phòng']:
        feature_cols.append(col)

X = df[feature_cols].fillna(0)
y = df['Giá bán (VND)']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✓ Train set: {len(X_train):,} samples")
print(f"✓ Test set: {len(X_test):,} samples")
print(f"✓ Features: {len(feature_cols)}")
print(f"\nFeature columns: {feature_cols}")

## 15. Kết Luận

### ✅ Đã hoàn thành:
1. Load và backup dữ liệu gốc
2. Làm sạch dữ liệu (empty rows, duplicates)
3. Xử lý missing values
4. Chuyển đổi giá từ text sang số
5. Xử lý các cột số
6. Feature engineering (tạo features mới)
7. Encoding categorical features
8. Lưu dữ liệu đã xử lý

### 📊 Kết quả:
- **File gốc**: Không bị thay đổi ✓
- **File processed**: Sẵn sàng cho modeling ✓
- **File backup**: Đã lưu an toàn ✓

### 🎓 Bước tiếp theo:
Bạn có thể sử dụng `X_train`, `X_test`, `y_train`, `y_test` để train các models như:
- Linear Regression
- Random Forest
- XGBoost
- Neural Networks